En este cuaderno, sobre la base de datos de Habitaclia obtenida en 2024, obtendremos dos nuevas bases de datos para entrenar un modelo que nos acerque a medir la calidad a través del texto. 

1. Sólo texto - calidad del 1 al 10. 
2. Texto + enlace a fotos - calidad del 1 al 10.

In [10]:
import pandas as pd
import numpy as np

# Lista de participantes

In [11]:
nombres = [
    "Alsharif, Dareen",
    "José Carlos",
    "Gonzalo",
    "Izumi Noda, Henry Daisuke",
    "Lopez Lopez, Víctor Stheffano",
    "Peralta Carvallo, Loreto Beatriz",
    "Román Mantilla, María Alejandra",
    "Ruiz Amezquita, Tatiana",
    "Ruiz Acha, Kathia",
    "Tian, Shiduo (Leo)",
    "Vásquez Umaña, José Daniel"
]

# Texto

In [12]:
df1=pd.read_excel('data/a-amb+mad-marzo24.xlsx')
len(df1)

3020

In [13]:
df1.head(2)

,ID,Title,Link,precio_euros,codigo_inmueble,Description,last_mod,dummy_indice,indice_alquiler,area,...,Lon_X,multifam,atico,duplex,estudio,loft,piso,triplex,AMB,MAD
0,1,Alquiler Piso en Carrer del consell de cent 451,https://www.habitaclia.com/alquiler-piso-calle...,1250,Anuncio: 49444000000010,OPORTUNIDAD EN EIXAMPLE DRETA!PISO ESPECIALMEN...,27/02/2024,0,0.0,85,...,2.174846,1,0,0,0,0,1,0,1,0
1,2,Alquiler Piso en Avinguda de cerdanyola 47. ...,https://www.habitaclia.com/alquiler-piso-moder...,1250,Anuncio: 1903004403789,Bonito y moderno piso de alquiler de dos habit...,06/03/2024,0,0.0,68,...,2.090375,1,0,0,0,0,1,0,1,0


In [14]:
df1['C_contempo']=None
df1['C_estado']=None
df1['C_armarios']=None

df1['B_contempo']=None
df1['B_estado']=None
df1['B_lavamano']=None

df1['R_contempo']=None
df1['R_estado']=None
df1['R_carpinteria']=None
df1['R_singular']=None
df1['R_ventana']=None

df1['dumi_precio_reduccion']=None
df1['Precio_reduccion']=None
df1['Anuncio_invalido']=None

In [15]:
print(df1.columns.tolist())

['ID', 'Title', 'Link', 'precio_euros', 'codigo_inmueble', 'Description', 'last_mod', 'dummy_indice', 'indice_alquiler', 'area', 'n_rooms', 'price_m2', 'n_baths', 'n_aseo', 'laundry', 'buhardilla', 'despacho', 'trastero', 'terrace', 'kwhm2_year', 'kgCO2m2_year', 'dummy_consum_EPC', 'dummy_emission_EPC', 'consum_A', 'consum_B', 'consum_C', 'consum_D', 'consum_E', 'consum_F', 'consum_G', 'emission_A', 'emission_B', 'emission_C', 'emission_D', 'emission_E', 'emission_F', 'emission_G', 'AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'barrio', 'codigo_inmueble_numero', 'Lat_Y', 'Lon_X', 'multifam', 'atico', 'duplex', 'estudio', 'loft', 'piso', 'triplex', 'AMB', 'MAD', 'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido']


In [16]:
df2=df1[['ID', 'Title', 'Link',
         'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 
         'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana', 'dumi_precio_reduccion', 
         'Precio_reduccion', 'Anuncio_invalido']]

In [17]:
df2['fquem'] = None
df2['luzart'] = None
df2['mdeag'] = None

C:\Users\Usuari\AppData\Local\Temp\ipykernel_8908\1556412764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fquem'] = None
C:\Users\Usuari\AppData\Local\Temp\ipykernel_8908\1556412764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['luzart'] = None
C:\Users\Usuari\AppData\Local\Temp\ipykernel_8908\1556412764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [19]:
# Ordenar la lista de nombres según la columna 'IDtext'
nombres_ordenados = sorted(nombres)

# Calcular el tamaño de cada parte
part_size = len(df2) // len(nombres_ordenados)
remainder = len(df2) % len(nombres_ordenados)

# Dividir df1 en partes
partes_df = np.split(df1.sample(frac=1).sort_values(by='ID'), np.cumsum([part_size] * (len(nombres_ordenados) - 1) + [part_size + remainder]))

# Asignar los nombres a cada parte
for i, parte in enumerate(partes_df):
    if i < len(nombres_ordenados):
        parte['Responsable texto'] = nombres_ordenados[i]
    else:
        parte['Responsable texto'] = nombres_ordenados[-1]  # Asignar el último nombre en caso de que haya más partes que nombres

# Concatenar las partes nuevamente en un solo DataFrame
df2 = pd.concat(partes_df)

In [21]:
df2['qprecio'] = pd.qcut(df2['precio_euros'], 5, labels=False)

In [22]:
df2.to_excel('data/eva_tex-amb+mad-marzo24.xlsx', index=False)

Para no sesgar la calificación, vamos a darles diferentes muestras entre la evaluación del texto y de las imágenes. Para ello creamos dos IDs: `IDtext` y `IDimg`. Aquí creo el primero, que es exactamente igual al ID original del dataframe.

In [7]:
df1['IDtext']=df1['ID']

Y ahora asigno responsables.

La tabla se vería así.

In [9]:
df1[['IDtext','Description','cal_texto','Responsable texto']].head(3)

,IDtext,Description,cal_texto,Responsable texto
0,1,OPORTUNIDAD EN EIXAMPLE DRETA!PISO ESPECIALMEN...,None,"Alsharif, Dareen"
1,2,Bonito y moderno piso de alquiler de dos habit...,None,"Alsharif, Dareen"
2,3,"Piso de 100m², compuesto de 4 dormitorios (2 d...",None,"Alsharif, Dareen"


Por defecto ya está ordenado de menor a mayor según el `IDtext`.

In [10]:
df1[['IDtext','Description','cal_texto','Responsable texto']].to_excel('data/eva_tex-amb+mad-marzo24.xlsx', index=False)

Tras la revisión de la columna `Description` deberán indicar un valor del 1 al 10 en la columna `cal_texto` sobre la siguiente pregunta: *¿De qué tan buena calidad arquitectónica crees que es el piso en oferta?*. 

Si consideran que la descripción **no** ofrece suficiente información para dar un veredicto sobre la calidad del inmueble, indicar el valor 0.

# Imagen

In [11]:
df1['dum_ext']=None
df1['dum_render']=None
#df1['dum_no_varios']=None
df1['dum_no_link']=None
df1['cal_img']=None

La columna `IDimg`, será el resultado de multiplicar por 2 los valores pares de `IDtext`, y por 3 los valores impares.

In [12]:
df1['IDimg'] = df1['IDtext'].apply(lambda x: x * 2 if x % 2 == 0 else x * 3)

Ordeno de menor a mayor con base en esa nueva columna para exportar.

In [13]:
df1 = df1.sort_values(by='IDimg')

Y ahora asigno responsables.

In [14]:
# Ordenar la lista de nombres según la columna 'IDtext'
nombres_ordenados = sorted(nombres)

# Calcular el tamaño de cada parte
part_size = len(df1) // len(nombres_ordenados)
remainder = len(df1) % len(nombres_ordenados)

# Dividir df1 en partes
partes_df = np.split(df1.sample(frac=1).sort_values(by='IDimg'), np.cumsum([part_size] * (len(nombres_ordenados) - 1) + [part_size + remainder]))

# Asignar los nombres a cada parte
for i, parte in enumerate(partes_df):
    if i < len(nombres_ordenados):
        parte['Responsable img'] = nombres_ordenados[i]
    else:
        parte['Responsable img'] = nombres_ordenados[-1]  # Asignar el último nombre en caso de que haya más partes que nombres

# Concatenar las partes nuevamente en un solo DataFrame
df1 = pd.concat(partes_df)

Para esta etapa deberán revisar las fotografías del interior del inmueble para completar la columna `cal_img` en  una escala del 1 al 10 con base en la siguiente pregunta: *¿De qué tan buena calidad arquitectónica crees que es el piso en oferta?* 

Si consideran que las imágenes **no** ofrecen suficiente información para dar un veredicto sobre la calidad del inmueble, indicar el valor 0.

Únicamente **los casos que no cumplan con tener al menos una fotografía del interior** deberán considerar las siguientes situaciones:
1. Si hay fotos del exterior del inmueble, indicar 1 en la columna `dum_ext`. Si no es el caso, dejarla vacía.
2. Si hay renders, indicar 1 en la columna `dum_render`. Si no es el caso, dejarla vacía.
3. Si el enlace está obsoleto indicar 1 en la columna `dum_no_link`. Si no es el caso, dejarla vacía.

Reviso la cabecera.

In [15]:
df1[['IDimg','cal_img','dum_ext','dum_render','dum_no_link','Responsable img']].head(3)

,IDimg,cal_img,dum_ext,dum_render,dum_no_link,Responsable img
0,3,None,None,None,None,"Alsharif, Dareen"
1,4,None,None,None,None,"Alsharif, Dareen"
3,8,None,None,None,None,"Alsharif, Dareen"


In [16]:
df1[['IDimg','Link','cal_img','dum_ext','dum_render',
     'dum_no_link','Responsable img']].to_excel(
    'data/eva_img-amb+mad-marzo24.xlsx',
    index=False)

<mark>Consultar con Carlos</mark>: 

1. Podríamos reducir `dum_ext`, `dum_render` y `dum_no_link` a una sola columna que sea `dum_invalid`.
2. Los chicos deberán crear una carpeta por cada *IDimg* y cargar en ella un mínimo de dos y un máximo de cinco imágenes. El enlace ya está preparado en mi drive personal: https://drive.google.com/drive/folders/1kWvQSJK-Ogy1ZC1iV9HhoAK73mHVgLSe?usp=sharing.
3. Si encuentran algún duplicado (que no debería existir, pero por si las dudas) deberán pintar la columna *Link* con amarillo.
4. ¿Haremos participar a Gonzalo? ¿Nosotros?